##Data Loading

In [3]:
#!/bin/bash!
!kaggle datasets download xhlulu/140k-real-and-fake-faces

Dataset URL: https://www.kaggle.com/datasets/xhlulu/140k-real-and-fake-faces
License(s): other
100% 3.73G/3.75G [00:52<00:00, 123MB/s]
100% 3.75G/3.75G [00:53<00:00, 75.8MB/s]


In [5]:
import zipfile
import os

zip_path = "/content/140k-real-and-fake-faces.zip"

extract_to = "/content/data"

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Unzipped files are extracted to {extract_to}")


KeyboardInterrupt: 

In [6]:
import cv2
import os
import numpy as np
import pandas as pd
df=pd.read_csv('/content/data/train.csv')

In [7]:
df_test=pd.read_csv('/content/data/test.csv')

In [8]:
df_test = df_test.sample(frac=1, random_state=32)

In [4]:
df = df.sample(frac=1, random_state=32)

In [9]:
data_dir='/content/data'
x_test=[]
j=0
for i in df_test['path']:
  if j==100:
    break
  image=cv2.imread(os.path.join(data_dir,i))
  image=cv2.resize(image,(256,256))
  image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
  x_test.append(image)
  j=j+1



In [6]:
data_dir='/content/data'
x=[]
j=0
for i in df['path']:
  if j==7000:
    break
  image=cv2.imread(os.path.join(data_dir,i))
  image=cv2.resize(image,(256,256))
  image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
  x.append(image)
  j=j+1



In [10]:
j=0
y_test=[]
for i in df_test['label']:
  if j==100:
    break
  y_test.append(i)
  j=j+1


In [8]:
j=0
y=[]
for i in df['label']:
  if j==7000:
    break
  y.append(i)
  j=j+1


In [9]:
import numpy as np
x=np.array(x)
y=np.array(y)

In [11]:
x_test=np.array(x_test)
y_test=np.array(y_test)

##Model Building

In [11]:
import tensorflow as tf
model=tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(256,256,3)),
    tf.keras.layers.Conv2D(8,(3,3),activation ='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Conv2D(8,(5,5),activation ='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Conv2D(16,(5,5),activation ='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Conv2D(16,(5,5),activation ='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16),

    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(1,activation ='sigmoid')

])

In [12]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

##Training the Model

In [17]:
model.fit(x,y,epochs=100,batch_size=32,validation_data=(x_test,y_test))

Epoch 1/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9841 - loss: 0.0410 - val_accuracy: 0.9000 - val_loss: 0.2917
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.9838 - loss: 0.0444 - val_accuracy: 0.9300 - val_loss: 0.2695
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9863 - loss: 0.0414 - val_accuracy: 0.9100 - val_loss: 0.4282
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.9857 - loss: 0.0381 - val_accuracy: 0.9400 - val_loss: 0.2372
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.9852 - loss: 0.0400 - val_accuracy: 0.9100 - val_loss: 0.3048
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9872 - loss: 0.0360 - val_accuracy: 0.9300 - val_loss: 0.3327
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.9889 - loss: 0.0360 - val_accuracy: 0.9400 - val_loss: 0.3108
Epoch 8/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.9891 - loss: 0.0371 

##Saving the Model

In [19]:
model.save('/content/model.keras')

In [20]:
model.save('model.h5')


##Loading the Model

In [1]:
import tensorflow as tf
loaded_model = tf.keras.models.load_model('/content/model.h5')


##Making the Predictions

In [53]:
predictions=loaded_model.predict(x_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


#Comparing the results with the original values

In [88]:
int(predictions[25]*100)

<ipython-input-88-52d3238f1e9d>:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  int(predictions[25]*100)


0

In [72]:
y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1])